Classificação de Texto Multilabel


José Augusto de Almeida Neto


## Importar bibliotecas


In [9]:
import os
import time
import pandas as pd
import torch
from dotenv import load_dotenv
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer

## Carregar datasets


In [10]:
# Carregar dataset
df_case_study = pd.read_csv('../datasets/dataset-case_study-tratado.csv')
df_case_study

,restaurante_name,user_name,review_date,stars,comment_text,year,platform,language,comment_text_translated
0,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,Hotelaria e gastronomia são duas artes demanda...
1,cipriani,Аннa Шкаленкова,11 dias atrás no Google,1,It was possible to order only a tasting menu;...,2024,Google,en,Foi possível encomendar apenas um menu de degu...
2,cipriani,Roberto Barretto,um mês atrás no Google,1,Grade 10! Food: 5 Service: 4 Atmosph...,Unknown,Google,en,Grade 10!
3,cipriani,GRUPO VANDERSON SOARES,um mês atrás no Google,1,Wonderful Food: 5 Service: 5 Atmosp...,Unknown,Google,en,comida maravilhosa
4,cipriani,Maria Baptista,um mês atrás no Google,1,We had a great dining experience at Cipriani....,Unknown,Google,en,Tivemos uma ótima experiência gastronômica em ...
...,...,...,...,...,...,...,...,...,...
4966,tangara,Christopher Lopez,4 anos atrás no Google,1,Great place for lunch. Nice scenary good wine ...,2020,Google,en,Ótimo lugar para o almoço.Lista de vinhos de b...
4967,tangara,Silvio Marcos,4 anos atrás no Google,1,Boa comida. Sabor marcante,2020,Google,pt,Boa comida. Sabor marcante
4968,tangara,Maria Lins,4 anos atrás no Google,1,Comida divina,2020,Google,it,comida divina
4969,tangara,Ricardo Polisel Alves,5 anos atrás no Foursquare,0,Tasting Menu was indeed very good. Service wa...,2019,Foursquare,en,O menu de degustação foi realmente muito bom.O...


In [11]:
labels = ['ambiente', 'bebida', 'comida', 'geral',
          'localização', 'outros', 'preço', 'serviço']

In [12]:
# Renomeando a coluna de df
df = df_case_study[['comment_text_translated']].copy()
df.rename(columns={'comment_text_translated': 'sentenca'}, inplace=True)
df

,sentenca
0,Hotelaria e gastronomia são duas artes demanda...
1,Foi possível encomendar apenas um menu de degu...
2,Grade 10!
3,comida maravilhosa
4,Tivemos uma ótima experiência gastronômica em ...
...,...
4966,Ótimo lugar para o almoço.Lista de vinhos de b...
4967,Boa comida. Sabor marcante
4968,comida divina
4969,O menu de degustação foi realmente muito bom.O...


## Hiperparâmetros


In [13]:
# Hiperparâmetros
max_len = 128
test_batch_size = 4
num_labels = len(labels)
threshold = 0.28

## Funções auxiliares


In [14]:
class TokenizeDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['sentenca']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten()
        }

In [15]:
def run_prediction(model, df_test):
    start_time = time.time()

    # Carregar o tokenizador BERT
    tokenizer = BertTokenizer.from_pretrained(
        'neuralmind/bert-base-portuguese-cased')

    # Tokenizar o dataset
    tokenized_test_data = TokenizeDataset(df_test, tokenizer, max_len)

    # Criar DataLoader para carregar os dados em lotes durante o treinamento
    test_loader = DataLoader(tokenized_test_data,
                              batch_size=test_batch_size, shuffle=True)

    model.eval()
    all_predictions = []
    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        for batch in test_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']

            # Passagem Direta (inferência)
            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            # A função sigmóide é aplicada aos logits para converter os valores em probabilidades
            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            # Aplicação do threshold
            predicted_labels = (predicted_probs > threshold).float()

            # Conversão dos rótulos previstos para numpy arrays
            predicted_labels_np = predicted_labels.cpu().numpy()

            # Coleção das predictions
            all_predictions.extend(predicted_labels_np)
    
    # Calcular o tempo total de teste
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)

    print(
        f"\nTempo de predição: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

    return all_predictions

## Aplicação do modelo

In [16]:
# Importar o modelo
model_dir = './bertimbau-model'
model = BertForSequenceClassification.from_pretrained(model_dir)

In [17]:
name = 'teste1'

# Fazer previsões no dataset sem anotações
predictions = run_prediction(model, df)

# Salvar as previsões em um arquivo CSV
predictions_df = pd.DataFrame(predictions, columns=labels)
predictions_df.to_csv(f'../datasets/dataset-case_study_predictions-{name}.csv', index=False)


Tempo de predição: 00:15:0.21
